In [89]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pickle
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score


In [90]:
df_combined = pd.read_csv('../Data/dataset_combinado.csv')

# Seleccionar las columnas que serán utilizadas como características (X) y la columna objetivo (y)
X = df_combined[['Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)', 
                 'Freedom', 'Trust (Government Corruption)', 'Generosity']]
y = df_combined['Happiness Score']


In [91]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [92]:
dt_model = DecisionTreeRegressor(random_state=42)


In [93]:
# Entrenar el modelo de Árbol de Decisión
dt_model.fit(X_train, y_train)


DecisionTreeRegressor(random_state=42)

In [94]:
# Hacer predicciones en el conjunto de prueba
y_pred_dt = dt_model.predict(X_test)


In [95]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [3, 5, 10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=DecisionTreeRegressor(random_state=42), 
                           param_grid=param_grid, 
                           cv=5, 
                           n_jobs=-1, 
                           verbose=2)

grid_search.fit(X_train, y_train)

best_dt_model = grid_search.best_estimator_

y_pred_best_dt = best_dt_model.predict(X_test)

r2_best_dt = r2_score(y_test, y_pred_best_dt)
mse_best_dt = mean_squared_error(y_test, y_pred_best_dt)

print(f'El modelo ajustado de Árbol de Decisión tiene una precisión del {r2_best_dt * 100:.2f}%')
print(f'Error Cuadrático Medio (MSE): {mse_best_dt:.4f}')


Fitting 5 folds for each of 45 candidates, totalling 225 fits
El modelo ajustado de Árbol de Decisión tiene una precisión del 67.71%
Error Cuadrático Medio (MSE): 0.4031


In [96]:
path = dt_model.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas = path.ccp_alphas  

train_scores = []
test_scores = []

for ccp_alpha in ccp_alphas:
    dt_pruned = DecisionTreeRegressor(random_state=42, ccp_alpha=ccp_alpha)
    dt_pruned.fit(X_train, y_train)
    
    train_scores.append(dt_pruned.score(X_train, y_train))
    test_scores.append(dt_pruned.score(X_test, y_test))

best_alpha = ccp_alphas[test_scores.index(max(test_scores))]

best_dt_pruned = DecisionTreeRegressor(random_state=42, ccp_alpha=best_alpha)
best_dt_pruned.fit(X_train, y_train)

y_pred_pruned = best_dt_pruned.predict(X_test)
r2_pruned = r2_score(y_test, y_pred_pruned)
mse_pruned = mean_squared_error(y_test, y_pred_pruned)

print(f'El modelo podado tiene una precisión del {r2_pruned * 100:.2f}%')
print(f'Error Cuadrático Medio (MSE): {mse_pruned:.4f}')


El modelo podado tiene una precisión del 68.17%
Error Cuadrático Medio (MSE): 0.3975


In [97]:
df_combined_encoded = pd.get_dummies(df_combined, columns=['Region'])

X_encoded = df_combined_encoded[['Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)', 
                                 'Freedom', 'Trust (Government Corruption)', 'Generosity'] + 
                                list(df_combined_encoded.columns[df_combined_encoded.columns.str.startswith('Region_')])]
y = df_combined_encoded['Happiness Score']

X_train_encoded, X_test_encoded, y_train_encoded, y_test_encoded = train_test_split(X_encoded, y, test_size=0.3, random_state=42)

best_dt_model.fit(X_train_encoded, y_train_encoded)

y_pred_encoded = best_dt_model.predict(X_test_encoded)

r2_encoded = r2_score(y_test_encoded, y_pred_encoded)
mse_encoded = mean_squared_error(y_test_encoded, y_pred_encoded)

print(f'El modelo con One-Hot Encoding tiene una precisión del {r2_encoded * 100:.2f}%')
print(f'Error Cuadrático Medio (MSE): {mse_encoded:.4f}')


El modelo con One-Hot Encoding tiene una precisión del 70.01%
Error Cuadrático Medio (MSE): 0.3744


In [99]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# Crear un ensamble con Árbol de Decisión y otros modelos
estimators = [
    ('dt', DecisionTreeRegressor(random_state=42)),
    ('rf', RandomForestRegressor(random_state=42))
]

stack_model = StackingRegressor(estimators=estimators, final_estimator=LinearRegression())

stack_model.fit(X_train, y_train)

y_pred_stack = stack_model.predict(X_test)

r2_stack = r2_score(y_test, y_pred_stack)
mse_stack = mean_squared_error(y_test, y_pred_stack)

print(f'El modelo en ensamble tiene una precisión del {r2_stack * 100:.2f}%')
print(f'Error Cuadrático Medio (MSE): {mse_stack:.4f}')


El modelo en ensamble tiene una precisión del 77.73%
Error Cuadrático Medio (MSE): 0.2781


In [101]:
import pickle

with open('../Model/decision_tree_model.pkl', 'wb') as file:
    pickle.dump(best_dt_model, file)


# Conclusiones sobre el modelo de Árbol de Decisión

## Comparativa de los modelos

A continuación, se presentan los diferentes resultados obtenidos con el modelo de Árbol de Decisión aplicando distintas técnicas de mejora:

1. **Modelo Inicial:**
   - Precisión: **67.71%**
   - Error Cuadrático Medio (MSE): **0.4031**

2. **Modelo Podado:**
   - Precisión: **68.17%**
   - Error Cuadrático Medio (MSE): **0.3975**

3. **Modelo con One-Hot Encoding:**
   - Precisión: **70.01%**
   - Error Cuadrático Medio (MSE): **0.3744**

4. **Modelo en Ensamble:**
   - Precisión: **77.73%**
   - Error Cuadrático Medio (MSE): **0.2781**

## Conclusiones

- La precisión del modelo mejoró progresivamente con la poda, la codificación One-Hot y el ensamble.
- El modelo con el enfoque de **Ensamble** mostró la mayor precisión, alcanzando un **77.73%**, lo cual representa un aumento significativo respecto al modelo original.
- El **Error Cuadrático Medio (MSE)** también disminuyó considerablemente, lo que indica que el modelo ensamble tiene una mejor capacidad para realizar predicciones con menor error.

